In [16]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import *
import numpy as np
from itertools import combinations
from sklearn.model_selection import PredefinedSplit
from sklearn.preprocessing import OneHotEncoder

import glob
pd.set_option('display.max_rows', None)

In [56]:
# Initialze the estimators
clf1 = RandomForestClassifier(random_state=42)
# clf2 = SVC(kernel='sigmoid',probability=True, random_state=42)
clf3 = LogisticRegression(random_state=42)
clf4 = DecisionTreeClassifier(random_state=42)
clf5 = KNeighborsClassifier()
clf6 = MultinomialNB()
# clf7 = GradientBoostingClassifier(random_state=42)
clf8 = MLPClassifier(max_iter=2000,random_state=42)
clf9 = SVC(kernel='poly',probability=True, random_state=42)
# clf10 = SVC(kernel='rbf',probability=True, random_state=42)


In [3]:
class ColumnExtractor(object):
    def __init__(self, cols=None):
        self.cols = cols
    def transform(self, X):
        return X[:,self.cols]
    def fit(self, X, y=None):
        return self
    def get_params(self,deep=True):
        return {"cols" : self.cols}
    
    def set_params(self, **parameters):
        for parameter, value in parameters.items():
            setattr(self, parameter, value)

In [4]:
# Initiaze the hyperparameters for each dictionary
param1 = {}
param1['feat_select__cols'] = sum([list(combinations(np.arange(0,4,1), i)) for i in range(1,5)], [])
param1['classifier__n_estimators'] = [10, 20, 30, 40]
param1['classifier__max_depth'] = [2, 3, 4]
param1['classifier__class_weight'] = [None, 'balanced']
# param1['classifier'] = [clf1]

param2 = {}
param2['feat_select__cols'] = sum([list(combinations(np.arange(0,4,1), i)) for i in range(1,5)], [])
param2['classifier__C'] = [10**-2, 10**-1, 10**0, 10**1, 10**2]
# param2['Classifier__kernel'] = [ 'linear']
param2['classifier__class_weight'] = [None, 'balanced']
param2['classifier__degree'] = [2,3]
# param2['classifier'] = [clf2]

param3 = {}
param3['feat_select__cols'] = sum([list(combinations(np.arange(0,4,1), i)) for i in range(1,5)], [])
param3['classifier__C'] = [10**-2, 10**-1, 10**0, 10**1, 10**2]
param3['classifier__penalty'] = ['l1', 'l2']
param3['classifier__class_weight'] = [None, 'balanced']
# param3['classifier'] = [clf3]

param4 = {}
param4['feat_select__cols'] = sum([list(combinations(np.arange(0,4,1), i)) for i in range(1,4)], [])
param4['classifier__max_depth'] = [2,3,4,None]
param4['classifier__min_samples_split'] = [2,3,4]
param4['classifier__class_weight'] = [None, 'balanced']
# param4['classifier'] = [clf4]

param5 = {}
param5['feat_select__cols'] = sum([list(combinations(np.arange(0,4,1), i)) for i in range(1,5)], [])
param5['classifier__n_neighbors'] = [2,3,4,5]
# param5['classifier'] = [clf5]

param6 = {}
param6['feat_select__cols'] = sum([list(combinations(np.arange(0,4,1), i)) for i in range(1,5)], [])
param6['classifier__alpha'] = [10**0, 10**1, 10**2]
# param6['classifier'] = [clf6]

param7 = {}
param7['feat_select__cols'] = sum([list(combinations(np.arange(0,4,1), i)) for i in range(1,5)], [])
param7['classifier__n_estimators'] = [10, 20, 30, 40]
param7['classifier__max_depth'] = [2, 3, 4]
# param7['classifier'] = [clf7]

param8 = {}
param8['feat_select__cols'] = sum([list(combinations(np.arange(0,4,1), i)) for i in range(1,5)], [])
param8['classifier__hidden_layer_sizes'] = [(10),(10,10),(10,10,10)]
param8['classifier__solver'] = ['adam', 'sgd']
param8['classifier__alpha'] = [0.0001, 0.0005,0.001,0.005,0.01,0.05]    
# param8['classifier'] = [clf8]

In [93]:
pipeline1 = Pipeline([('feat_select',ColumnExtractor()),('classifier', clf1)])
# pipeline2 = Pipeline([('feat_select',ColumnExtractor()),('classifier', clf2)])
pipeline3 = Pipeline([('feat_select',ColumnExtractor()),('classifier', clf3)])
pipeline4 = Pipeline([('feat_select',ColumnExtractor()),('classifier', clf4)])
pipeline5 = Pipeline([('feat_select',ColumnExtractor()),('classifier', clf5)])
pipeline6 = Pipeline([('feat_select',ColumnExtractor()),('classifier', clf6)])
# pipeline7 = Pipeline([('feat_select',ColumnExtractor()),('classifier', clf7)])
pipeline8 = Pipeline([('feat_select',ColumnExtractor()),('classifier', clf8)])
pipeline9 = Pipeline([('feat_select',ColumnExtractor()),('classifier', clf9)])
# pipeline10 = Pipeline([('feat_select',ColumnExtractor()),('classifier', clf10)])


In [94]:
rootPath = "./"
X_train = pd.read_csv(rootPath + "X_train_tenengrad.csv")
X_test = pd.read_csv(rootPath + "X_test_tenengrad.csv")
y_train = pd.read_csv(rootPath + "y_train_tenengrad.csv").values.reshape(-1)
y_test = pd.read_csv(rootPath + "y_test_tenengrad.csv").values.reshape(-1)

In [95]:
indices = np.zeros(len(y_train) + len(y_test))
indices[:len(y_train)] = -1
Data_X = np.vstack([X_train,X_test])
Data_Y = np.hstack([y_train,y_test])

ps = PredefinedSplit(indices)
score = { "Accuracy":"balanced_accuracy" ,"Recall":"recall_weighted","Precision":"precision_weighted","F1_Score":"f1_weighted"}

In [96]:
grid1 = GridSearchCV(pipeline1, n_jobs=-1, param_grid=[param1],scoring=score,refit="F1_Score",cv=ps)
# grid2 = GridSearchCV(pipeline2, n_jobs=-1, param_grid=[param2],scoring=score,refit="F1_Score",cv=ps)
grid3 = GridSearchCV(pipeline3, n_jobs=-1, param_grid=[param3],scoring=score,refit="F1_Score",cv=ps)
grid4 = GridSearchCV(pipeline4, n_jobs=-1, param_grid=[param4],scoring=score,refit="F1_Score",cv=ps)
grid5 = GridSearchCV(pipeline5, n_jobs=-1, param_grid=[param5],scoring=score,refit="F1_Score",cv=ps)
grid6 = GridSearchCV(pipeline6, n_jobs=-1, param_grid=[param6],scoring=score,refit="F1_Score",cv=ps)
# grid7 = GridSearchCV(pipeline7, n_jobs=-1, param_grid=[param7],scoring=score,refit="F1_Score",cv=ps)
grid8 = GridSearchCV(pipeline8, n_jobs=-1, param_grid=[param8],scoring=score,refit="F1_Score",cv=ps)
grid9 = GridSearchCV(pipeline9, n_jobs=-1, param_grid=[param2],scoring=score,refit="F1_Score",cv=ps)
# grid10 = GridSearchCV(pipeline10, n_jobs=-1, param_grid=[param2],scoring=score,refit="F1_Score",cv=ps)

In [97]:

grids = [grid1,grid3,grid4,grid5,grid6,grid8,grid9]

In [98]:
for pipe in grids:
    print("Yes Done")
    pipe.fit(Data_X,Data_Y)  

Yes Done
Yes Done


C:\Users\ronakdediya\anaconda33\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
150 fits failed out of a total of 300.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
150 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\ronakdediya\anaconda33\lib\site-packages\sklearn\model_selection\_validation.py", line 681, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\ronakdediya\anaconda33\lib\site-packages\sklearn\pipeline.py", line 394, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "C:\Users\ronakdediya\anaconda33\lib\site-packages\sklearn\linear_model\_logistic.py", line 1461, in fit
    solver = _check_sol

Yes Done
Yes Done
Yes Done
Yes Done
Yes Done


In [120]:
grid = grids[6]
print(grid.best_estimator_)
# print(pd.DataFrame(grid.cv_results_).iloc[grid.best_index_])

Pipeline(steps=[('feat_select',
                 <__main__.ColumnExtractor object at 0x000001AAD1CD82B0>),
                ('classifier',
                 SVC(C=0.1, kernel='poly', probability=True, random_state=42))])


In [121]:
df = pd.DataFrame(grid.cv_results_).drop(columns=["mean_fit_time","std_fit_time","mean_score_time","std_score_time","params",
                                             "mean_test_Accuracy","std_test_Accuracy","rank_test_Accuracy",
                                            "mean_test_Recall","std_test_Recall","rank_test_Recall",
                                            "mean_test_Precision","std_test_Precision","rank_test_Precision",
                                            "mean_test_F1_Score","std_test_F1_Score","rank_test_F1_Score"])

In [114]:
df["sum"] = df["split0_test_Accuracy"] + df["split0_test_Recall"] + df["split0_test_Precision"] + df["split0_test_F1_Score"]

In [122]:
df.sort_values("split0_test_Accuracy")[-10:][::-1]

,param_classifier__C,param_classifier__class_weight,param_classifier__degree,param_feat_select__cols,split0_test_Accuracy,split0_test_Recall,split0_test_Precision,split0_test_F1_Score
149,1,None,3,"(0, 1, 2, 3)",0.457143,0.457143,0.445714,0.418022
178,1,balanced,3,"(1, 2, 3)",0.457143,0.457143,0.468333,0.416752
296,100,balanced,3,"(0, 1, 3)",0.457143,0.457143,0.425556,0.409697
115,0.1,balanced,3,"(0, 1, 2)",0.457143,0.457143,0.445714,0.418022
179,1,balanced,3,"(0, 1, 2, 3)",0.457143,0.457143,0.445714,0.418022
85,0.1,None,3,"(0, 1, 2)",0.457143,0.457143,0.445714,0.418022
266,100,None,3,"(0, 1, 3)",0.457143,0.457143,0.425556,0.409697
148,1,None,3,"(1, 2, 3)",0.457143,0.457143,0.468333,0.416752
100,0.1,balanced,2,"(0, 1, 2)",0.428571,0.428571,0.412381,0.387253
274,100,balanced,2,"(0, 1)",0.428571,0.428571,0.391270,0.388394


In [12]:
import pandas as pd
pd.read_csv("time_hwt.csv").iloc[0]

0     [92.95910811424255]
1      [82.2305908203125]
2     [82.43215727806091]
3     [80.79285311698914]
4      [80.9026141166687]
             ...         
85    [78.49931454658508]
86     [77.7010612487793]
87    [77.35790419578552]
88    [76.98056769371033]
89    [76.88145112991333]
Name: 0, Length: 90, dtype: object

In [ ]:
17

In [ ]:
95sec